In [1]:
from __future__ import print_function

import os
import time
import numpy as np
import tensorflow as tf
import pandas as pd
from collections import defaultdict

from sklearn.metrics import roc_auc_score, accuracy_score
import nltk

from correct_text import train, decode, decode_sentence, evaluate_accuracy, create_model,\
    get_corrective_tokens, DefaultPTBConfig, DefaultMovieDialogConfig
from text_corrector_data_readers import PTBDataReader, MovieDialogReader

%matplotlib inline

In [2]:
root_data_path = "datasets/cornell movie-dialogs corpus"
train_path = os.path.join(root_data_path, "movie_lines.txt0000")
val_path = os.path.join(root_data_path, "movie_lines.txt0001")
test_path = os.path.join(root_data_path, "movie_lines.txt0002")
model_path = os.path.join(root_data_path, "dialog_correcter_model_testnltk")
config = DefaultMovieDialogConfig()

## Train

In [3]:
data_reader = MovieDialogReader(config, train_path)

In [ ]:
train(data_reader, train_path, val_path, model_path)

Reading data; train = datasets/cornell movie-dialogs corpus/movie_lines.txt0000, test = datasets/cornell movie-dialogs corpus/movie_lines.txt0001


W0914 19:23:38.341596 140050958817088 deprecation_wrapper.py:119] From /media/catarina/DATA/git/deep-text-corrector/correct_text.py:143: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0914 19:23:38.378669 140050958817088 deprecation_wrapper.py:119] From /media/catarina/DATA/git/deep-text-corrector/text_corrector_models.py:74: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0914 19:23:38.432886 140050958817088 deprecation_wrapper.py:119] From /media/catarina/DATA/git/deep-text-corrector/text_corrector_models.py:108: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0914 19:23:38.433595 140050958817088 deprecation.py:506] From /home/catarina/anaconda3/envs/deep-learning/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instruction

Creating 4 layers of 512 units.


W0914 19:23:38.799648 140050958817088 deprecation.py:506] From /home/catarina/anaconda3/envs/deep-learning/lib/python3.7/site-packages/tensorflow/python/ops/rnn_cell_impl.py:738: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0914 19:23:39.406693 140050958817088 deprecation.py:506] From /home/catarina/anaconda3/envs/deep-learning/lib/python3.7/site-packages/tensorflow/contrib/rnn/python/ops/core_rnn_cell.py:183: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


[TensorShape([Dimension(None), Dimension(1), Dimension(512)]), TensorShape([Dimension(None), Dimension(1), Dimension(512)]), TensorShape([Dimension(None), Dimension(1), Dimension(512)]), TensorShape([Dimension(None), Dimension(1), Dimension(512)]), TensorShape([Dimension(None), Dimension(1), Dimension(512)]), TensorShape([Dimension(None), Dimension(1), Dimension(512)]), TensorShape([Dimension(None), Dimension(1), Dimension(512)]), TensorShape([Dimension(None), Dimension(1), Dimension(512)]), TensorShape([Dimension(None), Dimension(1), Dimension(512)]), TensorShape([Dimension(None), Dimension(1), Dimension(512)])]
[TensorShape([Dimension(None), Dimension(1), Dimension(512)]), TensorShape([Dimension(None), Dimension(1), Dimension(512)]), TensorShape([Dimension(None), Dimension(1), Dimension(512)]), TensorShape([Dimension(None), Dimension(1), Dimension(512)]), TensorShape([Dimension(None), Dimension(1), Dimension(512)]), TensorShape([Dimension(None), Dimension(1), Dimension(512)]), Tensor

W0914 19:23:52.851113 140050958817088 deprecation_wrapper.py:119] From /media/catarina/DATA/git/deep-text-corrector/text_corrector_models.py:194: The name tf.trainable_variables is deprecated. Please use tf.compat.v1.trainable_variables instead.

W0914 19:23:52.851612 140050958817088 deprecation_wrapper.py:119] From /media/catarina/DATA/git/deep-text-corrector/text_corrector_models.py:199: The name tf.train.GradientDescentOptimizer is deprecated. Please use tf.compat.v1.train.GradientDescentOptimizer instead.

W0914 19:23:56.612573 140050958817088 deprecation.py:323] From /home/catarina/anaconda3/envs/deep-learning/lib/python3.7/site-packages/tensorflow/python/ops/clip_ops.py:286: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0914 19:24:25.659644 140050958817088 deprecation_wrapper.py:119] From /media/catari

Created model with fresh parameters.
Training bucket sizes: [48, 80306, 55100, 56500]
Total train size: 191954.0


## Decode sentences

In [ ]:
data_reader = MovieDialogReader(config, train_path, dropout_prob=0.25, replacement_prob=0.25, dataset_copies=1)

In [ ]:
corrective_tokens = get_corrective_tokens(data_reader, train_path)

In [ ]:
import pickle
with open(os.path.join(root_data_path, "corrective_tokens.pickle"), "w") as f:
    pickle.dump(corrective_tokens, f)

In [ ]:
import pickle
with open(os.path.join(root_data_path, "token_to_id.pickle"), "w") as f:
    pickle.dump(data_reader.token_to_id, f)

In [ ]:
sess = tf.InteractiveSession()
model = create_model(sess, True, model_path, config=config)

In [ ]:
# Test a sample from the test dataset.
decoded = decode_sentence(sess, model, data_reader, "you must have girlfriend", corrective_tokens=corrective_tokens)

In [ ]:
decoded

In [ ]:
decoded = decode_sentence(sess, model, data_reader,
                          "did n't you say that they 're going to develop this revolutionary new thing ...",
                          corrective_tokens=corrective_tokens)

In [ ]:
decode_sentence(sess, model, data_reader, "kvothe went to market", corrective_tokens=corrective_tokens, verbose=False)

In [ ]:
decode_sentence(sess, model, data_reader, "blablahblah and bladdddd went to market", corrective_tokens=corrective_tokens,
                verbose=False)

In [ ]:
decode_sentence(sess, model, data_reader, "do you have book", corrective_tokens=corrective_tokens, verbose=False)

In [ ]:
decode_sentence(sess, model, data_reader, "the cardinals did better then the cubs", corrective_tokens=corrective_tokens, verbose=False)

In [ ]:
# 4 layers, 40k steps
errors = evaluate_accuracy(sess, model, data_reader, corrective_tokens, test_path)#, max_samples=1000)

In [ ]:
# 4 layers, 30k steps
errors = evaluate_accuracy(sess, model, data_reader, corrective_tokens, test_path)#, max_samples=1000)

In [ ]:
# 4 layers, 20k steps
errors = evaluate_accuracy(sess, model, data_reader, corrective_tokens, test_path)#, max_samples=1000)

In [ ]:
errors = evaluate_accuracy(sess, model, data_reader, corrective_tokens, test_path)#, max_samples=1000)

In [ ]:
for decoding, target in errors:
    print("Decoding: " + " ".join(decoding))
    print("Target:   " + " ".join(target) + "\n")